In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression

In [3]:

# Cargar el dataset
file_path = 'dataset/safta.csv'
data = pd.read_csv(file_path, quotechar='"', delimiter=";")

columns_to_drop = ['EMPRESA', 'NUMERO_FACTURA', 'NUMERO_ASIENTO_BORRADOR', 'TIPO_FACTURA', 'DIARIO', 'IMPORTE_COBRADO_FRA', 'CUENTA_CONTABLE','NUM_EFECTOS_COBRADOS', 'NUM_EFECTOS_PARCIAL', 'NUM_EFECTOS_IMPAGADO', 'NUM_EFECTOS_FUERA_PLAZO', 'NUM_EFECTOS_PDTE_EN_PLAZO']
data.drop(columns=columns_to_drop, inplace=True)

# Split the data into training and testing sets
train_data = data.sample(frac=0.8, random_state=0)
test_data = data.drop(train_data.index)
# load the data into an autogluon tabular dataset
dataset = TabularDataset(train_data)
label = 'CATEGORIA_FACTURA'

In [4]:
predictor = TabularPredictor(label=label).fit(dataset, time_limit=3600, presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20240615_174224"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.0
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.5.0: Wed May  1 20:13:18 PDT 2024; root:xnu-10063.121.3~5/RELEASE_ARM64_T6030
CPU Count:          11
Memory Avail:       14.69 GB / 36.00 GB (40.8%)
Disk Space Avail:   156.87 GB / 460.43 GB (34.1%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `

(_ray_fit pid=73529) [1000]	valid_set's multi_error: 0.0289815


(_dystack pid=73357) 	0.9697	 = Validation score   (accuracy)
(_dystack pid=73357) 	37.93s	 = Training   runtime
(_dystack pid=73357) 	15.87s	 = Validation runtime
(_dystack pid=73357) Fitting model: LightGBM_BAG_L1 ... Training model for up to 495.73s of the 794.37s of remaining time.
(_dystack pid=73357) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.75%)


(_ray_fit pid=73631) [1000]	valid_set's multi_error: 0.0266642 [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)


(_dystack pid=73357) 	0.9703	 = Validation score   (accuracy)
(_dystack pid=73357) 	25.67s	 = Training   runtime
(_dystack pid=73357) 	9.82s	 = Validation runtime
(_dystack pid=73357) Fitting model: RandomForestGini_BAG_L1 ... Training model for up to 465.74s of the 764.39s of remaining time.
(_dystack pid=73357) 	0.9699	 = Validation score   (accuracy)
(_dystack pid=73357) 	2.12s	 = Training   runtime
(_dystack pid=73357) 	1.12s	 = Validation runtime
(_dystack pid=73357) Fitting model: RandomForestEntr_BAG_L1 ... Training model for up to 462.39s of the 761.03s of remaining time.
(_dystack pid=73357) 	0.9699	 = Validation score   (accuracy)
(_dystack pid=73357) 	2.08s	 = Training   runtime
(_dystack pid=73357) 	1.14s	 = Validation runtime
(_dystack pid=73357) Fitting model: CatBoost_BAG_L1 ... Training model for up to 459.05s of the 757.69s of remaining time.
(_dystack pid=73357) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus

In [5]:
y_pred = predictor.predict(test_data.drop(columns=[label]))
y_pred.head()

2     1
10    2
13   -1
20   -1
21   -1
Name: CATEGORIA_FACTURA, dtype: int64

In [6]:
predictor.evaluate(test_data, silent=True)

{'accuracy': 0.9757211637381178,
 'balanced_accuracy': 0.664066459561783,
 'mcc': 0.9415588455864996}

In [7]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L2,0.976338,0.973869,accuracy,26.200131,94.742296,1758.917516,0.243994,0.510346,7.617027,2,True,20
1,LightGBMLarge_BAG_L2,0.975886,0.974692,accuracy,26.664507,96.155962,1771.530831,0.708370,1.924012,20.230342,2,True,28
2,NeuralNetFastAI_BAG_L2,0.975886,0.975073,accuracy,27.210413,95.079211,1809.567150,1.254276,0.847261,58.266661,2,True,18
3,XGBoost_BAG_L2,0.975721,0.975330,accuracy,26.739723,94.994199,1774.002132,0.783586,0.762249,22.701643,2,True,26
4,WeightedEnsemble_L3,0.975721,0.975330,accuracy,26.740650,94.998358,1776.734716,0.000927,0.004160,2.732584,3,True,35
5,LightGBM_r131_BAG_L2,0.975680,0.975052,accuracy,26.597577,96.013804,1768.772399,0.641440,1.781855,17.471910,2,True,31
6,NeuralNetFastAI_r191_BAG_L2,0.975474,0.974918,accuracy,27.880690,95.256975,1907.638899,1.924553,1.025025,156.338410,2,True,32
7,RandomForestEntr_BAG_L2,0.975392,0.974970,accuracy,26.095435,96.148463,1758.233031,0.139298,1.916513,6.932542,2,True,22
8,ExtraTreesGini_BAG_L2,0.975310,0.974918,accuracy,26.133498,96.386456,1753.668185,0.177361,2.154506,2.367696,2,True,24
9,CatBoost_r9_BAG_L2,0.975186,0.974785,accuracy,26.146872,94.391948,1880.805461,0.190735,0.159998,129.504972,2,True,33


In [8]:
predictor.feature_importance(test_data)

These features in provided data are not utilized by the predictor and will be ignored: ['CENTRO_CONTABLE']
Computing feature importance via permutation shuffling for 16 features using 5000 rows with 5 shuffle sets...
	634.79s	= Expected runtime (126.96s per shuffle set)
	476.75s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
MAX_FECHA_COBRO,0.38184,0.011615,1.026113e-07,5,0.405755,0.357925
FORMA_COBRO,0.03232,0.002823,6.918693e-06,5,0.038134,0.026506
FECHA_FACTURA,0.02504,0.001813,3.277005e-06,5,0.028774,0.021306
MES_FACTURA,0.00772,0.001006,3.383015e-05,5,0.009791,0.005649
CLIENTE,0.00608,0.001301,2.367650e-04,5,0.008758,0.003402
EJERCICIO,0.00108,0.000832,2.199146e-02,5,0.002793,-0.000633
ORGANIZACION_COMERCIAL,0.00108,0.000179,8.710929e-05,5,0.001448,0.000712
TIENE_DESCUENTO,0.00084,0.000385,4.073251e-03,5,0.001632,0.000048
NUM_EFECTOS_FACTURA,0.00060,0.000490,2.598985e-02,5,0.001609,-0.000409
DIVISA,-0.00008,0.000110,9.110961e-01,5,0.000146,-0.000306


In [9]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                          model  score_val eval_metric  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0                XGBoost_BAG_L2   0.975330    accuracy      94.994199  1774.002132                0.762249          22.701643            2       True         26
1           WeightedEnsemble_L3   0.975330    accuracy      94.998358  1776.734716                0.004160           2.732584            3       True         35
2        NeuralNetFastAI_BAG_L2   0.975073    accuracy      95.079211  1809.567150                0.847261          58.266661            2       True         18
3          LightGBM_r131_BAG_L2   0.975052    accuracy      96.013804  1768.772399                1.781855          17.471910            2       True         31
4       RandomForestGini_BAG_L2   0.975042    accuracy      96.162332  1760.811433                1.930382           9.510944        

/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/autogluon/core/utils/plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


In [10]:
predictor.info()

{'path': 'AutogluonModels/ag-20240615_174224',
 'label': 'CATEGORIA_FACTURA',
 'random_state': 0,
 'version': '1.1.1',
 'features': ['EJERCICIO',
  'NUMERO_SERIE',
  'ORGANIZACION_COMERCIAL',
  'CLIENTE',
  'FECHA_FACTURA',
  'MES_FACTURA',
  'DIVISA',
  'FORMA_COBRO',
  'LIQUIDO_FACTURA',
  'ALBARAN_FACTURA',
  'ENVIO_ELECTRONICO',
  'FECHA_CONTABILIZACION',
  'TIENE_DESCUENTO',
  'COUNT_TIPOS_TRANSACCION',
  'MAX_FECHA_COBRO',
  'NUM_EFECTOS_FACTURA'],
 'feature_metadata_in': <autogluon.common.features.feature_metadata.FeatureMetadata at 0x338da45b0>,
 'time_fit_preprocessing': 2.1935410499572754,
 'time_fit_training': 2681.02104306221,
 'time_fit_total': 2683.2145841121674,
 'time_limit': 2688.8111629486084,
 'time_train_start': 1718474258.340534,
 'num_rows_train': 97203,
 'num_cols_train': 26,
 'num_rows_val': None,
 'num_classes': 5,
 'problem_type': 'multiclass',
 'eval_metric': 'accuracy',
 'best_model': 'WeightedEnsemble_L3',
 'best_model_score_val': 0.9753299795273809,
 'best

In [11]:
#Limpiamos todas las variables para proceder con otro modelo, liberamos memoria
del data
del train_data
del test_data
del dataset
del predictor
del y_pred

# Cargar el dataset
file_path = 'dataset/vima.csv'
data = pd.read_csv(file_path, quotechar='"', delimiter=";")

columns_to_drop = ['EMPRESA', 'NUMERO_FACTURA', 'NUMERO_ASIENTO_BORRADOR', 'TIPO_FACTURA', 'DIARIO', 'IMPORTE_COBRADO_FRA', 'CUENTA_CONTABLE','NUM_EFECTOS_COBRADOS', 'NUM_EFECTOS_PARCIAL', 'NUM_EFECTOS_IMPAGADO', 'NUM_EFECTOS_FUERA_PLAZO', 'NUM_EFECTOS_PDTE_EN_PLAZO']
data.drop(columns=columns_to_drop, inplace=True)

# Split the data into training and testing sets
train_data = data.sample(frac=0.8, random_state=0)
test_data = data.drop(train_data.index)
# load the data into an autogluon tabular dataset
dataset = TabularDataset(train_data)
label = 'CATEGORIA_FACTURA'

/var/folders/xx/fj7dzt957rd1sg25bl4jv1b00000gn/T/ipykernel_73319/4234373678.py:11: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, quotechar='"', delimiter=";")


In [12]:
predictor = TabularPredictor(label=label).fit(dataset, time_limit=3600, presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20240615_185211"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.0
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.5.0: Wed May  1 20:13:18 PDT 2024; root:xnu-10063.121.3~5/RELEASE_ARM64_T6030
CPU Count:          11
Memory Avail:       14.93 GB / 36.00 GB (41.5%)
Disk Space Avail:   172.32 GB / 460.43 GB (37.4%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `

In [13]:
predictor.feature_importance(test_data)

These features in provided data are not utilized by the predictor and will be ignored: ['TIENE_DESCUENTO', 'NUM_EFECTOS_FACTURA']
Computing feature importance via permutation shuffling for 15 features using 5000 rows with 5 shuffle sets...
	5514.33s	= Expected runtime (1102.87s per shuffle set)
	4969.27s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
MAX_FECHA_COBRO,0.42844,0.009468,2.860353e-08,5,0.447935,0.408945
COUNT_TIPOS_TRANSACCION,0.29268,0.004728,8.167530e-09,5,0.302415,0.282945
FORMA_COBRO,0.24016,0.005680,3.753174e-08,5,0.251856,0.228464
MES_FACTURA,0.16080,0.002731,9.984991e-09,5,0.166424,0.155176
FECHA_FACTURA,0.09104,0.003425,2.398177e-07,5,0.098091,0.083989
FECHA_CONTABILIZACION,0.04456,0.001936,4.264886e-07,5,0.048546,0.040574
CLIENTE,0.01828,0.001254,2.639140e-06,5,0.020862,0.015698
EJERCICIO,0.01372,0.001718,2.890485e-05,5,0.017258,0.010182
ALBARAN_FACTURA,0.00060,0.000374,1.152518e-02,5,0.001370,-0.000170
DIVISA,0.00016,0.000639,3.026433e-01,5,0.001475,-0.001155


In [14]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val eval_metric  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L3   0.974387    accuracy    2388.284552  1762.807835                0.009701           5.168297            3       True         22
1            XGBoost_BAG_L2   0.973984    accuracy    2370.431600  1497.962951                1.720535          78.713408            2       True         20
2   RandomForestEntr_BAG_L2   0.973863    accuracy    2372.474998  1441.161789                3.763933          21.912246            2       True         16
3   RandomForestGini_BAG_L2   0.973836    accuracy    2372.549304  1446.269542                3.838239          27.019999            2       True         15
4         LightGBMXT_BAG_L2   0.973775    accuracy    2371.649633  1440.884078                2.938568          21.634535            2       True        

/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/autogluon/core/utils/plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


{'model_types': {'KNeighborsUnif_BAG_L1': 'StackerEnsembleModel_KNN',
  'KNeighborsDist_BAG_L1': 'StackerEnsembleModel_KNN',
  'NeuralNetFastAI_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'RandomForestGini_BAG_L1': 'StackerEnsembleModel_RF',
  'RandomForestEntr_BAG_L1': 'StackerEnsembleModel_RF',
  'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'ExtraTreesGini_BAG_L1': 'StackerEnsembleModel_XT',
  'ExtraTreesEntr_BAG_L1': 'StackerEnsembleModel_XT',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel',
  'NeuralNetFastAI_BAG_L2': 'StackerEnsembleModel_NNFastAiTabular',
  'LightGBMXT_BAG_L2': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L2': 'StackerEnsembleModel_LGB',
  'RandomForestGini_BAG_L2': 'StackerEnsembleModel_RF',
  'RandomForestEntr_BAG_L2': 'StackerEnsembleModel_RF',
  'CatBoost_BAG_L2': 'StackerEnsembleModel_CatBoost',
  'ExtraTreesGini_BAG_L2': 'StackerEnsembleModel_

In [15]:
predictor.info()

{'path': 'AutogluonModels/ag-20240615_185211',
 'label': 'CATEGORIA_FACTURA',
 'random_state': 0,
 'version': '1.1.1',
 'features': ['EJERCICIO',
  'NUMERO_SERIE',
  'ORGANIZACION_COMERCIAL',
  'CLIENTE',
  'FECHA_FACTURA',
  'MES_FACTURA',
  'DIVISA',
  'FORMA_COBRO',
  'CENTRO_CONTABLE',
  'LIQUIDO_FACTURA',
  'ALBARAN_FACTURA',
  'ENVIO_ELECTRONICO',
  'FECHA_CONTABILIZACION',
  'COUNT_TIPOS_TRANSACCION',
  'MAX_FECHA_COBRO'],
 'feature_metadata_in': <autogluon.common.features.feature_metadata.FeatureMetadata at 0x426ffc8b0>,
 'time_fit_preprocessing': 0.9816601276397705,
 'time_fit_training': 2615.875893831253,
 'time_fit_total': 2616.857553958893,
 'time_limit': 2611.5631330013275,
 'time_train_start': 1718478520.926288,
 'num_rows_train': 181506,
 'num_cols_train': 26,
 'num_rows_val': None,
 'num_classes': 5,
 'problem_type': 'multiclass',
 'eval_metric': 'accuracy',
 'best_model': 'WeightedEnsemble_L3',
 'best_model_score_val': 0.9743865216576862,
 'best_model_stack_level': 3,


In [16]:
y_pred = predictor.predict(test_data.drop(columns=[label]))
y_pred.head()

2     1
10    0
21    0
27    0
30    1
Name: CATEGORIA_FACTURA, dtype: int64

In [17]:
predictor.evaluate(test_data, silent=True)

{'accuracy': 0.9749647390691114,
 'balanced_accuracy': 0.9169399936191274,
 'mcc': 0.9642822347158182}

In [18]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.974965,0.974387,accuracy,562.174678,2388.284552,1762.807835,0.006593,0.009701,5.168297,3,True,22
1,LightGBMXT_BAG_L2,0.974877,0.973775,accuracy,556.883163,2371.649633,1440.884078,0.946098,2.938568,21.634535,2,True,13
2,NeuralNetFastAI_BAG_L2,0.974833,0.973219,accuracy,558.240847,2370.059539,1587.245930,2.303782,1.348475,167.996387,2,True,12
3,LightGBM_BAG_L2,0.974744,0.973582,accuracy,556.547835,2370.501553,1433.102166,0.610770,1.790488,13.852623,2,True,14
4,XGBoost_BAG_L2,0.974744,0.973984,accuracy,557.412979,2370.431600,1497.962951,1.475914,1.720535,78.713408,2,True,20
5,RandomForestEntr_BAG_L2,0.974612,0.973863,accuracy,556.208820,2372.474998,1441.161789,0.271755,3.763933,21.912246,2,True,16
6,ExtraTreesEntr_BAG_L2,0.974392,0.973736,accuracy,556.265198,2372.861006,1425.978547,0.328133,4.149941,6.729004,2,True,19
7,RandomForestGini_BAG_L2,0.974370,0.973836,accuracy,556.226301,2372.549304,1446.269542,0.289236,3.838239,27.019999,2,True,15
8,CatBoost_BAG_L2,0.974237,0.973714,accuracy,556.289290,2369.013530,1899.237981,0.352225,0.302465,479.988438,2,True,17
9,ExtraTreesGini_BAG_L2,0.974083,0.973665,accuracy,556.270531,2372.874613,1426.510340,0.333466,4.163548,7.260797,2,True,18


In [19]:
#Limpiamos todas las variables para proceder con otro modelo, liberamos memoria
del data
del train_data
del test_data
del dataset
del predictor
del y_pred

# Cargar el dataset
file_path = 'dataset/suco.csv'
data = pd.read_csv(file_path, quotechar='"', delimiter=";")

columns_to_drop = ['EMPRESA', 'NUMERO_FACTURA', 'NUMERO_ASIENTO_BORRADOR', 'TIPO_FACTURA', 'DIARIO', 'IMPORTE_COBRADO_FRA', 'CUENTA_CONTABLE','NUM_EFECTOS_COBRADOS', 'NUM_EFECTOS_PARCIAL', 'NUM_EFECTOS_IMPAGADO', 'NUM_EFECTOS_FUERA_PLAZO', 'NUM_EFECTOS_PDTE_EN_PLAZO']
data.drop(columns=columns_to_drop, inplace=True)

# Split the data into training and testing sets
train_data = data.sample(frac=0.8, random_state=0)
test_data = data.drop(train_data.index)
# load the data into an autogluon tabular dataset
dataset = TabularDataset(train_data)
label = 'CATEGORIA_FACTURA'

/var/folders/xx/fj7dzt957rd1sg25bl4jv1b00000gn/T/ipykernel_73319/2381697021.py:11: DtypeWarning: Columns (2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, quotechar='"', delimiter=";")


In [20]:
predictor = TabularPredictor(label=label).fit(dataset, time_limit=3600, presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20240615_214402"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.0
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.5.0: Wed May  1 20:13:18 PDT 2024; root:xnu-10063.121.3~5/RELEASE_ARM64_T6030
CPU Count:          11
Memory Avail:       12.85 GB / 36.00 GB (35.7%)
Disk Space Avail:   165.82 GB / 460.43 GB (36.0%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `

In [21]:
predictor.info()

{'path': 'AutogluonModels/ag-20240615_214402',
 'label': 'CATEGORIA_FACTURA',
 'random_state': 0,
 'version': '1.1.1',
 'features': ['EJERCICIO',
  'NUMERO_SERIE',
  'ORGANIZACION_COMERCIAL',
  'CLIENTE',
  'FECHA_FACTURA',
  'MES_FACTURA',
  'DIVISA',
  'FORMA_COBRO',
  'CENTRO_CONTABLE',
  'LIQUIDO_FACTURA',
  'ALBARAN_FACTURA',
  'ENVIO_ELECTRONICO',
  'FECHA_CONTABILIZACION',
  'TIENE_DESCUENTO',
  'COUNT_TIPOS_TRANSACCION',
  'MAX_FECHA_COBRO',
  'NUM_EFECTOS_FACTURA'],
 'feature_metadata_in': <autogluon.common.features.feature_metadata.FeatureMetadata at 0x426bc58d0>,
 'time_fit_preprocessing': 2.5173532962799072,
 'time_fit_training': 2613.0544719696045,
 'time_fit_total': 2615.5718252658844,
 'time_limit': 2614.2410781383514,
 'time_train_start': 1718488830.3670862,
 'num_rows_train': 448744,
 'num_cols_train': 23,
 'num_rows_val': None,
 'num_classes': 4,
 'problem_type': 'multiclass',
 'eval_metric': 'accuracy',
 'best_model': 'WeightedEnsemble_L3',
 'best_model_score_val': 0

In [22]:
predictor.feature_importance(test_data)

Computing feature importance via permutation shuffling for 17 features using 5000 rows with 5 shuffle sets...
	3066.8s	= Expected runtime (613.36s per shuffle set)
	2792.08s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
MAX_FECHA_COBRO,0.41096,0.006550,7.743057e-09,5,0.424447,0.397473
FECHA_FACTURA,0.12792,0.003132,4.311172e-08,5,0.134370,0.121470
FORMA_COBRO,0.10668,0.003895,2.128939e-07,5,0.114700,0.098660
MES_FACTURA,0.09740,0.003367,1.711905e-07,5,0.104334,0.090466
DIVISA,0.05844,0.003814,2.165132e-06,5,0.066293,0.050587
CLIENTE,0.03092,0.005228,9.444749e-05,5,0.041685,0.020155
EJERCICIO,0.02896,0.002670,8.570712e-06,5,0.034457,0.023463
TIENE_DESCUENTO,0.02224,0.001220,1.081703e-06,5,0.024752,0.019728
ORGANIZACION_COMERCIAL,0.01684,0.002188,3.345085e-05,5,0.021345,0.012335
CENTRO_CONTABLE,0.01320,0.001393,1.465781e-05,5,0.016068,0.010332


In [23]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val eval_metric  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L3   0.960784    accuracy    3334.588135  1446.009773                0.018530           7.118403            3       True         15
1   RandomForestEntr_BAG_L2   0.960690    accuracy    3312.417305  1369.036056                7.518397          36.208539            2       True         10
2   RandomForestGini_BAG_L2   0.960508    accuracy    3312.574502  1374.996655                7.675594          42.169138            2       True          9
3     ExtraTreesGini_BAG_L2   0.960109    accuracy    3311.687748  1345.327808                6.788840          12.500291            2       True         12
4     ExtraTreesEntr_BAG_L2   0.960104    accuracy    3312.586774  1348.013402                7.687866          15.185885            2       True        

/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/autogluon/core/utils/plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


{'model_types': {'KNeighborsUnif_BAG_L1': 'StackerEnsembleModel_KNN',
  'KNeighborsDist_BAG_L1': 'StackerEnsembleModel_KNN',
  'NeuralNetFastAI_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel',
  'NeuralNetFastAI_BAG_L2': 'StackerEnsembleModel_NNFastAiTabular',
  'LightGBMXT_BAG_L2': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L2': 'StackerEnsembleModel_LGB',
  'RandomForestGini_BAG_L2': 'StackerEnsembleModel_RF',
  'RandomForestEntr_BAG_L2': 'StackerEnsembleModel_RF',
  'CatBoost_BAG_L2': 'StackerEnsembleModel_CatBoost',
  'ExtraTreesGini_BAG_L2': 'StackerEnsembleModel_XT',
  'ExtraTreesEntr_BAG_L2': 'StackerEnsembleModel_XT',
  'XGBoost_BAG_L2': 'StackerEnsembleModel_XGBoost',
  'WeightedEnsemble_L3': 'WeightedEnsembleModel'},
 'model_performance': {'KNeighborsUnif_BAG_L1': 0.8524882783948087,
  'KNeighborsDist_BAG_L1': 0.8622822812115594,
  'NeuralNetFastAI_BAG_L1': 0.9333205569322376

In [24]:
y_pred = predictor.predict(test_data.drop(columns=[label]))
y_pred.head()

1    -1
4     1
22    0
27   -1
30    0
Name: CATEGORIA_FACTURA, dtype: int64

In [25]:
predictor.evaluate(test_data, silent=True)

{'accuracy': 0.9610379191699499,
 'balanced_accuracy': 0.9342776735012341,
 'mcc': 0.9291415663074092}

In [26]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestGini_BAG_L2,0.961100,0.960508,accuracy,685.968874,3312.574502,1374.996655,0.564127,7.675594,42.169138,2,True,9
1,WeightedEnsemble_L3,0.961038,0.960784,accuracy,687.570592,3334.588135,1446.009773,0.006730,0.018530,7.118403,3,True,15
2,RandomForestEntr_BAG_L2,0.960904,0.960690,accuracy,685.926392,3312.417305,1369.036056,0.521645,7.518397,36.208539,2,True,10
3,ExtraTreesGini_BAG_L2,0.960779,0.960109,accuracy,685.894979,3311.687748,1345.327808,0.490232,6.788840,12.500291,2,True,12
4,NeuralNetFastAI_BAG_L2,0.960771,0.958515,accuracy,688.253433,3306.616293,1478.144111,2.848686,1.717385,145.316594,2,True,6
5,ExtraTreesEntr_BAG_L2,0.960690,0.960104,accuracy,685.987858,3312.586774,1348.013402,0.583111,7.687866,15.185885,2,True,13
6,CatBoost_BAG_L2,0.960093,0.958794,accuracy,685.985630,3305.453754,1696.295011,0.580883,0.554846,363.467494,2,True,11
7,LightGBM_BAG_L2,0.960040,0.958685,accuracy,686.449664,3307.686000,1357.107551,1.044916,2.787092,24.280034,2,True,8
8,LightGBMXT_BAG_L2,0.959897,0.958217,accuracy,690.376132,3322.835614,1428.374114,4.971385,17.936707,95.546597,2,True,7
9,XGBoost_BAG_L2,0.959754,0.958284,accuracy,687.602512,3307.891512,1371.927160,2.197765,2.992604,39.099643,2,True,14


In [27]:
#Limpiamos todas las variables para proceder con otro modelo, liberamos memoria
del data
del train_data
del test_data
del dataset
del predictor
del y_pred

# Cargar el dataset
file_path = 'dataset/decepal.csv'
data = pd.read_csv(file_path, quotechar='"', delimiter=";")

columns_to_drop = ['EMPRESA', 'NUMERO_FACTURA', 'NUMERO_ASIENTO_BORRADOR', 'TIPO_FACTURA', 'DIARIO', 'IMPORTE_COBRADO_FRA', 'CUENTA_CONTABLE','NUM_EFECTOS_COBRADOS', 'NUM_EFECTOS_PARCIAL', 'NUM_EFECTOS_IMPAGADO', 'NUM_EFECTOS_FUERA_PLAZO', 'NUM_EFECTOS_PDTE_EN_PLAZO']
data.drop(columns=columns_to_drop, inplace=True)

# Split the data into training and testing sets
train_data = data.sample(frac=0.8, random_state=0)
test_data = data.drop(train_data.index)
# load the data into an autogluon tabular dataset
dataset = TabularDataset(train_data)
label = 'CATEGORIA_FACTURA'

/var/folders/xx/fj7dzt957rd1sg25bl4jv1b00000gn/T/ipykernel_73319/2004628554.py:11: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, quotechar='"', delimiter=";")


In [28]:
predictor = TabularPredictor(label=label).fit(dataset, time_limit=3600, presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20240616_000542"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.0
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.5.0: Wed May  1 20:13:18 PDT 2024; root:xnu-10063.121.3~5/RELEASE_ARM64_T6030
CPU Count:          11
Memory Avail:       12.66 GB / 36.00 GB (35.2%)
Disk Space Avail:   160.40 GB / 460.43 GB (34.8%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `

In [29]:
predictor.feature_importance(test_data)

These features in provided data are not utilized by the predictor and will be ignored: ['DIVISA', 'TIENE_DESCUENTO']
Computing feature importance via permutation shuffling for 15 features using 5000 rows with 5 shuffle sets...
	3706.53s	= Expected runtime (741.31s per shuffle set)
	3287.8s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
COUNT_TIPOS_TRANSACCION,0.25932,0.007356,7.761730e-08,5,0.274466,0.244174
FECHA_FACTURA,0.21388,0.007632,1.942608e-07,5,0.229595,0.198165
NUM_EFECTOS_FACTURA,0.11736,0.005471,5.649357e-07,5,0.128624,0.106096
NUMERO_SERIE,0.08032,0.003068,2.549212e-07,5,0.086637,0.074003
MAX_FECHA_COBRO,0.04892,0.004431,7.987837e-06,5,0.058043,0.039797
MES_FACTURA,0.02344,0.003204,4.088629e-05,5,0.030038,0.016842
LIQUIDO_FACTURA,0.02092,0.003116,5.738651e-05,5,0.027337,0.014503
CLIENTE,0.01560,0.003507,2.869317e-04,5,0.022821,0.008379
ORGANIZACION_COMERCIAL,0.01088,0.002373,2.552295e-04,5,0.015766,0.005994
FORMA_COBRO,0.01032,0.002644,4.748697e-04,5,0.015765,0.004875


In [30]:
predictor.info()

{'path': 'AutogluonModels/ag-20240616_000542',
 'label': 'CATEGORIA_FACTURA',
 'random_state': 0,
 'version': '1.1.1',
 'features': ['EJERCICIO',
  'NUMERO_SERIE',
  'ORGANIZACION_COMERCIAL',
  'CLIENTE',
  'FECHA_FACTURA',
  'MES_FACTURA',
  'FORMA_COBRO',
  'CENTRO_CONTABLE',
  'LIQUIDO_FACTURA',
  'ALBARAN_FACTURA',
  'ENVIO_ELECTRONICO',
  'FECHA_CONTABILIZACION',
  'COUNT_TIPOS_TRANSACCION',
  'MAX_FECHA_COBRO',
  'NUM_EFECTOS_FACTURA'],
 'feature_metadata_in': <autogluon.common.features.feature_metadata.FeatureMetadata at 0x420fb05e0>,
 'time_fit_preprocessing': 4.444716930389404,
 'time_fit_training': 2574.1156709194183,
 'time_fit_total': 2578.5603878498077,
 'time_limit': 2573.874460220337,
 'time_train_start': 1718497372.994485,
 'num_rows_train': 237458,
 'num_cols_train': 25,
 'num_rows_val': None,
 'num_classes': 5,
 'problem_type': 'multiclass',
 'eval_metric': 'accuracy',
 'best_model': 'WeightedEnsemble_L3',
 'best_model_score_val': 0.9404063034305014,
 'best_model_stac

In [31]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val eval_metric  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L3   0.940406    accuracy    2469.228814  1521.841810                0.011432           5.749812            3       True         20
1   RandomForestGini_BAG_L2   0.939699    accuracy    2465.708681  1405.959595                3.670780          41.058583            2       True         17
2   RandomForestEntr_BAG_L2   0.939198    accuracy    2464.400169  1386.868807                2.362269          21.967795            2       True         18
3    NeuralNetFastAI_BAG_L2   0.933757    accuracy    2463.184333  1453.065620                1.146432          88.164608            2       True         14
4           LightGBM_BAG_L2   0.932628    accuracy    2499.137904  1865.530601               37.100003         500.629589            2       True        

/Users/bonay/.pyenv/versions/3.10.0/envs/gal_lua/lib/python3.10/site-packages/autogluon/core/utils/plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


{'model_types': {'KNeighborsUnif_BAG_L1': 'StackerEnsembleModel_KNN',
  'KNeighborsDist_BAG_L1': 'StackerEnsembleModel_KNN',
  'NeuralNetFastAI_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'RandomForestGini_BAG_L1': 'StackerEnsembleModel_RF',
  'RandomForestEntr_BAG_L1': 'StackerEnsembleModel_RF',
  'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'ExtraTreesGini_BAG_L1': 'StackerEnsembleModel_XT',
  'ExtraTreesEntr_BAG_L1': 'StackerEnsembleModel_XT',
  'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'LightGBMLarge_BAG_L1': 'StackerEnsembleModel_LGB',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel',
  'NeuralNetFastAI_BAG_L2': 'StackerEnsembleModel_NNFastAiTabular',
  'LightGBMXT_BAG_L2': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L2': 'StackerEnsembleModel_LGB',
  'RandomForestGini_BAG_L2': 'StackerEnsembleModel_RF',
  'RandomForestEntr_BAG_L2': 'StackerEnsembleModel_RF

In [32]:
y_pred = predictor.predict(test_data.drop(columns=[label]))
y_pred.head()

4     1
10    1
21    1
27    1
35    1
Name: CATEGORIA_FACTURA, dtype: int64

In [33]:
predictor.evaluate(test_data, silent=True)

{'accuracy': 0.9351964153358938,
 'balanced_accuracy': 0.8545430771625254,
 'mcc': 0.8911487382519306}

In [34]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,NeuralNetFastAI_BAG_L2,0.935651,0.933757,accuracy,482.517035,2463.184333,1453.065620,2.005087,1.146432,88.164608,2,True,14
1,WeightedEnsemble_L3,0.935196,0.940406,accuracy,483.023973,2469.228814,1521.841810,0.003782,0.011432,5.749812,3,True,20
2,LightGBM_BAG_L2,0.934961,0.932628,accuracy,496.356169,2499.137904,1865.530601,15.844221,37.100003,500.629589,2,True,16
3,RandomForestEntr_BAG_L2,0.934691,0.939198,accuracy,480.712054,2464.400169,1386.868807,0.200106,2.362269,21.967795,2,True,18
4,RandomForestGini_BAG_L2,0.934371,0.939699,accuracy,480.814997,2465.708681,1405.959595,0.303049,3.670780,41.058583,2,True,17
5,LightGBMXT_BAG_L2,0.933933,0.931954,accuracy,483.992448,2475.075849,1497.216561,3.480500,13.037949,132.315549,2,True,15
6,LightGBM_BAG_L1,0.928509,0.922946,accuracy,96.394357,468.790924,429.788313,96.394357,468.790924,429.788313,1,True,5
7,CatBoost_BAG_L2,0.928458,0.926492,accuracy,480.726203,2462.179490,1375.237170,0.214255,0.141590,10.336158,2,True,19
8,WeightedEnsemble_L2,0.928374,0.926539,accuracy,465.663821,2403.646687,967.836987,0.004043,0.011657,4.014141,2,True,13
9,LightGBMXT_BAG_L1,0.924668,0.922416,accuracy,368.830575,1930.181273,523.053435,368.830575,1930.181273,523.053435,1,True,4
